In [2]:
%reset -f

In [3]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from clustering import run_clustering


/home/syamagami/anaconda3/envs/generic/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
N_CUDA = 0
PARENT = 'Office31'
DATA_DOMAIN = 'amazon_dslr_webcam'
log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)

==============  amazon_dslr_webcam  ==============


In [5]:
config.config_path = CONFIG_FILE
config.log_dir = log_dir
config.lap = 1
# feats, edls_dataset = run_clustering(config)

In [6]:
# 正解率
dfs = []
dfs.append(pd.read_csv(os.path.join(log_dir, 'real_domain.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'cluster_pca_gmm.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'real_class.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'cluster_pca_gmm_class.csv'), names=['label'], dtype=int))

df = pd.concat(dfs, axis=1)
df.columns = ['rdomain', 'edomain', 'rclass', 'eclass']
df

,rdomain,edomain,rclass,eclass
0,0,1,0,8
1,0,1,0,8
2,0,1,0,8
3,0,1,0,8
4,0,1,0,8
...,...,...,...,...
4105,2,2,30,19
4106,2,1,30,22
4107,2,1,30,22
4108,2,2,30,12


In [7]:
print(  NMI(df.rdomain, df.edomain)  )
print(  100.0 * len(df[df.rdomain == df.edomain]) / len(df)  )
print(  NMI(df.rclass, df.eclass)  )
print(  100.0 * len(df[df.rclass == df.eclass]) / len(df)  )

0.4322815476803166
43.84428223844282
0.18310308839325073
2.5547445255474455


In [20]:
from sklearn.metrics import confusion_matrix

cmx = pd.DataFrame(confusion_matrix(df.rdomain, df.edomain), index = ['r_amazon', 'r_dslr', 'r_webcam'],  columns = ['e_amazon', 'e_dslr', 'e_webcam'])
cmx['total'] = cmx.sum(axis=1)
display(cmx)


,e_amazon,e_dslr,e_webcam,total
r_amazon,1332,1452,33,2817
r_dslr,0,6,492,498
r_webcam,6,325,464,795
